# 3 Spark DataFrame y Spark SQL



### como se creaba un RDD
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Spark1").getOrCreate()

<b>rdd = spark.sparkContext.parallelize([1,2,3,4,5,6])
</b>

## como creamos un Dataframe
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Spark3").getOrCreate()

<b>df = spark.read.options(header='True', inferSchema='True').csv("./datos/2019-Nov.csv")
</b>



- ejemplo de Data Frame y Spark SQL con python


- Descargamos los datos de una tienda de cosmetico 2,5GB partidos en 5 ficheros 
https://www.kaggle.com/mkechinov/ecommerce-events-history-in-cosmetics-shop




In [11]:

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Spark3").getOrCreate()
print(spark)



df = spark.read.options(header='True', inferSchema='True').csv("./datos/*.csv")





In [12]:
#contamos el numero de registros
df.count()

20692840

In [13]:
#sacamos la estructura de los datos
df.printSchema()

root
 |-- event_time: string (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- category_id: long (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- user_session: string (nullable = true)



In [15]:
#mostramos los primeros elementos
#filter , head

df.head(2)
df.filter("event_type='cart'").filter("brand='runail'").head(2)

[Row(event_time='2019-12-01 00:00:02 UTC', event_type='cart', product_id=4958, category_id=1487580009471148064, category_code=None, brand='runail', price=1.19, user_id=494077766, user_session='c99a50e8-2fac-4c4d-89ec-41c05f114554'),
 Row(event_time='2019-12-01 00:00:34 UTC', event_type='cart', product_id=5335, category_id=1487580009605365797, category_code=None, brand='runail', price=0.4, user_id=494077766, user_session='c99a50e8-2fac-4c4d-89ec-41c05f114554')]

In [20]:
#buscamos todos los tipos de eventos
#select, distinct


tipos_eventos= df.select("event_type").distinct()


df.select("event_type").distinct().show()




+----------------+
|      event_type|
+----------------+
|        purchase|
|            view|
|            cart|
|remove_from_cart|
+----------------+



In [7]:
#buscamos todas las marcas distintas

marcas_distintas = df.select('brand').distinct()
marcas_distintas.show()


+------------+
|       brand|
+------------+
|     beautix|
|     farmona|
|  dr.gloderm|
|   profhenna|
|    bioderma|
|     philips|
|invisibobble|
|       riche|
|        nova|
|        oniq|
|    lebelage|
|     vilenta|
|       fancy|
|      jaguar|
|      tertio|
|    siberina|
|   koreatida|
|         jas|
|rocknailstar|
|   depilflax|
+------------+
only showing top 20 rows



In [27]:
#buscamos todas sesiones 
seasons = df.select('user_session').distinct()
seasons.count()



4535942

In [38]:
#al ser una tienda y tener listado de compras, podemos sacar listados de produyctos relacionados

#primero scamos dotos los podructos que se hayan comprado

compras = df.select('product_id').where("event_type='purchase'").distinct()
compras.show()
compras.count()

+----------+
|product_id|
+----------+
|   5815137|
|   5670719|
|   5574996|
|   5900643|
|   5863096|
|   5834157|
|   5823267|
|   5868397|
|   5809871|
|   5814871|
|   5885863|
|     59990|
|   5837927|
|   5695639|
|   5767927|
|   5859171|
|   5563822|
|   5744098|
|   5908237|
|   5890174|
+----------+
only showing top 20 rows



40777

In [42]:
# dado el producto 5844397
# obtengo todas las sesiones de compra que contengan dicho producto

#buscamos los procutos que se hayan comprado en dichas sesiones
#isin

sessions=df.select(["user_session"]).filter("event_type='cart'AND product_id = 5844397").distinct()
#sessions=df.select(["user_session"]).filter("event_type='cart'").filter("product_id = 5844397").distinct()
sessions.count()
productos = df.select(["product_id"]).filter("event_type='cart'AND product_id <> 5844397").filter(df["user_session"].isin(sessions["user_session"]))
productos.count()

5763384

In [44]:
productos.write.mode("overwrite").csv("./datos/products_related")

In [23]:
carrito = df.filter(col("event_type") == "cart")

marcas_por_carrito = carrito.groupBy("brand").count().orderBy(col("count").desc())

marcas_por_carrito.show()

+---------+-------+
|    brand|  count|
+---------+-------+
|     null|2493706|
|   runail| 491818|
|    irisk| 333316|
|   masura| 247038|
|  grattol| 217364|
|bpw.style| 132954|
| ingarden| 124240|
|    estel|  86678|
|   kapous|  72096|
|freedecor|  70450|
|      uno|  67375|
|  italwax|  65230|
|     pole|  57508|
|  bluesky|  57112|
| haruyama|  55183|
|     milv|  50832|
|    domix|  47257|
| nagaraku|  44856|
| jessnail|  43233|
|  concept|  42893|
+---------+-------+
only showing top 20 rows



# Con Data frame tambien podemos trabajar con la api de SQL

Las consultas SQL no funcionan con RDD

In [24]:
df.createOrReplaceTempView("data")
spark.sql("select * from data limit 3").show()

+--------------------+----------------+----------+-------------------+-------------+------+-----+---------+--------------------+
|          event_time|      event_type|product_id|        category_id|category_code| brand|price|  user_id|        user_session|
+--------------------+----------------+----------+-------------------+-------------+------+-----+---------+--------------------+
|2019-12-01 00:00:...|remove_from_cart|   5712790|1487580005268456287|         null| f.o.x| 6.27|576802932|51d85cb0-897f-48d...|
|2019-12-01 00:00:...|            view|   5764655|1487580005411062629|         null|   cnd|29.05|412120092|8adff31e-2051-489...|
|2019-12-01 00:00:...|            cart|      4958|1487580009471148064|         null|runail| 1.19|494077766|c99a50e8-2fac-4c4...|
+--------------------+----------------+----------+-------------------+-------------+------+-----+---------+--------------------+



In [25]:
spark.sql("select * from data where event_type='cart' limit 3").show()

+--------------------+----------+----------+-------------------+-------------+------+-----+---------+--------------------+
|          event_time|event_type|product_id|        category_id|category_code| brand|price|  user_id|        user_session|
+--------------------+----------+----------+-------------------+-------------+------+-----+---------+--------------------+
|2019-12-01 00:00:...|      cart|      4958|1487580009471148064|         null|runail| 1.19|494077766|c99a50e8-2fac-4c4...|
|2019-12-01 00:00:...|      cart|   5629988|1487580009311764506|         null|  null| 1.19|579966747|1512be50-d0fd-4a9...|
|2019-12-01 00:00:...|      cart|      5335|1487580009605365797|         null|runail|  0.4|494077766|c99a50e8-2fac-4c4...|
+--------------------+----------+----------+-------------------+-------------+------+-----+---------+--------------------+



In [30]:
spark.sql("select count(*) from data where event_type='cart'").show()

+--------+
|count(1)|
+--------+
| 5768333|
+--------+



In [29]:
spark.sql("select count(*) as total from data where event_type='cart'").show()

+-------+
|  total|
+-------+
|5768333|
+-------+



In [31]:
resultado_sql = spark.sql("""
    SELECT brand, COUNT(*) AS total
    FROM data
    WHERE event_type='cart'
    GROUP BY brand
""")


resultado_sql.show()


+------------+-----+
|       brand|total|
+------------+-----+
|     beautix|26468|
|     farmona| 1430|
|  dr.gloderm|  515|
|   profhenna| 2051|
|    bioderma|  120|
|     philips|   58|
|       riche|  527|
|        oniq|26562|
|    lebelage| 1237|
|     vilenta| 1613|
|       fancy|  357|
|    siberina| 3307|
|      jaguar| 1298|
|      tertio| 1755|
|   koreatida|   90|
|         jas| 3327|
|rocknailstar|   16|
|   depilflax| 8758|
|protokeratin|  801|
|     relouis| 2948|
+------------+-----+
only showing top 20 rows

